# ARIMA Model

Source:

ning.oreilly.com/library/view/machine-learning-for/9781119682363/c04.xhtml#head-2-16

An **ARMA method** consists of two parts:

- An autoregression
- A moving average model

Compared with the autoregressive and moving average models, ARMA models provide the most efficient linear model of stationary time series, since they are capable of modeling the unknown process with the minimum number of parameters (Zhang et al. 2015).

In particular, ARMA models are used to describe weekly stationary stochastic time series in terms of two polynomials. The first of these polynomials is for autoregression, the second for the moving average. Often this method is referred to as the ARMA(p,q) model, in which:

- p stands for the order of the autoregressive polynomial, and
- q stands for the order of the moving average polynomial.

Here we will see how to simulate time series from AR(p), MA(q), and ARMA(p,q) processes as well as fit time series models to data based on insights gathered from the ACF and PACF.

**Autoregressive integrated moving average (ARIMA)** models are considered a development of the simpler autoregressive moving average (ARMA) models and include the notion of integration.

The main differences between ARMA and ARIMA methods are the notions of integration and differencing. An ARMA model is a stationary model, and it works very well with stationary time series.

ARIMA models have three main components, denoted as p, d, q; 

- p stands for the number of lag variables included in the ARIMA model, also called the lag order.
- d stands for the number of times that the raw values in a time series data set are differenced, also called the degree of differencing.
- q denotes the magnitude of the moving average window, also called the order of moving average.

Let's now take a look at an extension of the ARIMA model in Python, called SARIMAX, which stands for seasonal autoregressive integrated moving average with exogenous factors. 

Data scientists usually apply SARIMAX when they have to deal with time series data sets that have seasonal cycles. 

SARIMAX models support seasonality and exogenous factors.

SARIMAX requires another set of p, d, and q arguments for the seasonality aspect as well as a parameter called s, which is the periodicity of the seasonal cycle in your time series data set.

In [1]:
# Import necessary libraries
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error
import math
import pandas as pd
import numpy as np

In [2]:
# Load data set
df = pd.read_csv('data/GEFCom2014-E.csv')
print(df.head())
df['Date'] = pd.to_datetime(df['Date'])
df['DateTime'] = df.apply(lambda row: row['Date'] + pd.Timedelta(hours=row['Hour']), axis=1)
df.set_index('DateTime', inplace=True)
df = df.rename_axis(None)
print(df)
df.shape

     Date  Hour  load      T
0  1/1/04     1   NaN  37.33
1  1/1/04     2   NaN  37.67
2  1/1/04     3   NaN  37.00
3  1/1/04     4   NaN  36.33
4  1/1/04     5   NaN  36.00


/var/folders/tj/cfj9qmvs7150dnbbhjnzz1tw0000gq/T/ipykernel_23492/1349270763.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'])


                          Date  Hour    load      T
2004-01-01 01:00:00 2004-01-01     1     NaN  37.33
2004-01-01 02:00:00 2004-01-01     2     NaN  37.67
2004-01-01 03:00:00 2004-01-01     3     NaN  37.00
2004-01-01 04:00:00 2004-01-01     4     NaN  36.33
2004-01-01 05:00:00 2004-01-01     5     NaN  36.00
...                        ...   ...     ...    ...
2014-12-31 20:00:00 2014-12-31    20  4012.0  18.00
2014-12-31 21:00:00 2014-12-31    21  3856.0  16.67
2014-12-31 22:00:00 2014-12-31    22  3671.0  17.00
2014-12-31 23:00:00 2014-12-31    23  3499.0  15.33
2015-01-01 00:00:00 2014-12-31    24  3345.0  15.33

[96432 rows x 4 columns]


(96432, 4)

In [3]:
train = df['T'].iloc[:95000]
test = df['T'].iloc[95000:]
print('Train data shape: ', train.shape)
print('Test data shape: ', test.shape)
train = train.to_frame()
test = test.to_frame()
print(train.head())
print(test.head())

Train data shape:  (95000,)
Test data shape:  (1432,)
                         T
2004-01-01 01:00:00  37.33
2004-01-01 02:00:00  37.67
2004-01-01 03:00:00  37.00
2004-01-01 04:00:00  36.33
2004-01-01 05:00:00  36.00
                         T
2014-11-02 09:00:00  36.33
2014-11-02 10:00:00  36.67
2014-11-02 11:00:00  36.67
2014-11-02 12:00:00  37.00
2014-11-02 13:00:00  37.33


In [4]:
# Scale train data to be in range (0, 1)
scaler = MinMaxScaler()
train['T'] = scaler.fit_transform(train)
train.head()
 
# Scale test data to be in range (0, 1)
test['T'] = scaler.transform(test)
test.head()

,T
2014-11-02 09:00:00,0.472435
2014-11-02 10:00:00,0.475391
2014-11-02 11:00:00,0.475391
2014-11-02 12:00:00,0.478261
2014-11-02 13:00:00,0.481130


In [5]:
# Specify the number of steps to forecast ahead
HORIZON = 3
print('Forecasting horizon:', HORIZON, 'hours')

Forecasting horizon: 3 hours


In [6]:
# Define the order and seasonal order for the SARIMAX model
order = (4, 1, 0)
seasonal_order = (1, 1, 0, 24)

In [7]:
# Build and fit the SARIMAX model
model = SARIMAX(endog=train, order=order, seasonal_order=seasonal_order)
results = model.fit()
 
print(results.summary())

/Users/mjack6/GSU_Spring2025/MSA8200/venv_timeseries/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency h will be used.
  self._init_dates(dates, freq)
/Users/mjack6/GSU_Spring2025/MSA8200/venv_timeseries/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency h will be used.
  self._init_dates(dates, freq)
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            6     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -3.04050D+00    |proj g|=  1.66766D+01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    6      4     16      1     0     0   2.390D-02  -3.054D+00
  F =  -3.0539107573303328     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
                                     SARIMAX Results                                      
Dep. Variable:                                  T   No. Observations:                95000
Model:      

In [8]:
# Walk-forward validation:
# Create a test data point for each HORIZON step
test_shifted = test.copy()
 
for t in range(1, HORIZON):
    test_shifted['T+'+str(t)] = test_shifted['T'].shift(-t, freq='H')
    
test_shifted = test_shifted.dropna(how='any')

/var/folders/tj/cfj9qmvs7150dnbbhjnzz1tw0000gq/T/ipykernel_23492/1499647554.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  test_shifted['T+'+str(t)] = test_shifted['T'].shift(-t, freq='H')
/var/folders/tj/cfj9qmvs7150dnbbhjnzz1tw0000gq/T/ipykernel_23492/1499647554.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  test_shifted['T+'+str(t)] = test_shifted['T'].shift(-t, freq='H')


We can make predictions on the test data and use a simpler model (by specifying a different order and seasonal order) for demonstration:

In [9]:
%%time
# Make predictions on the test data
training_window = 720
 
train_ts = train['T']
test_ts = test_shifted
 
history = [x for x in train_ts]
history = history[(-training_window):]
 
predictions = []
 
# Let's user simpler model
order = (2, 1, 0)
seasonal_order = (1, 1, 0, 24)
 
for t in range(test_ts.shape[0]):
    model = SARIMAX(endog=history, order=order, seasonal_order=seasonal_order)
    model_fit = model.fit()
    yhat = model_fit.forecast(steps = HORIZON)
    predictions.append(yhat)
    obs = list(test_ts.iloc[t])
    # move the training window
    history.append(obs[0])
    history.pop(0)
    print(test_ts.index[t])
    print(t+1, ': predicted =', yhat, 'expected =', obs)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -2.92381D+00    |proj g|=  1.49805D+01


 This problem is unconstrained.

 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.



At iterate    5    f= -2.93491D+00    |proj g|=  4.98142D-01

At iterate   10    f= -2.93569D+00    |proj g|=  2.14325D-01

At iterate   15    f= -2.93573D+00    |proj g|=  8.82148D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    4     15     55      2     0     0   8.821D-03  -2.936D+00
  F =  -2.9357277061806637     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
2014-11-02 09:00:00
1 : predicted = [0.47797027 0.49934446 0.51449708] expected = [0.47243478260869565, 0.47539130434782617, 0.47539130434782617]



   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -2.92396D+00    |proj g|=  1.47224D+01

At iterate    5    f= -2.93459D+00    |proj g|=  1.04471D-01

At iterate   10    f= -2.93518D+00    |proj g|=  2.56427D+00

At iterate   15    f= -2.93558D+00    |proj g|=  9.25712D-02

At iterate   20    f= -2.93559D+00    |proj g|=  3.47817D-02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    4     23     52      1     0     0   6.384D-02  -2.936D+00
  F =  -2.9355892846123082     

CONVERG

...


 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.



           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    4     33     92      2     0     0   3.886D-03  -2.940D+00
  F =  -2.9396712627083956     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
2014-11-04 14:00:00
54 : predicted = [0.62866816 0.63950041 0.64381952] expected = [0.6232173913043478, 0.6347826086956522, 0.6347826086956522]
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -2.92743D+00    |proj g|=  1.44123D+01



   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.
 This problem is unconstrained.



At iterate    5    f= -2.93747D+00    |proj g|=  9.76209D-02

At iterate   10    f= -2.93817D+00    |proj g|=  3.84236D+00

At iterate   15    f= -2.93952D+00    |proj g|=  1.40592D-02


KeyboardInterrupt: 

In [ ]:
# Compare predictions to actual temperature
eval_df = pd.DataFrame(predictions, columns=['t+'+str(t) for t in range(1, HORIZON+1)])
eval_df['timestamp'] = test.index[0:len(test.index)-HORIZON+1]
eval_df = pd.melt(eval_df, id_vars='timestamp', 
value_name='prediction', var_name='h')
eval_df['actual'] = np.array(np.transpose(test_ts)).ravel()
eval_df[['prediction', 'actual']] = scaler.inverse_transform(eval_df[['prediction', 'actual']])

In [ ]:
# Compute the mean absolute percentage error (MAPE)
if(HORIZON> 1):
    eval_df['APE'] = (eval_df['prediction'] - 
        eval_df['actual']).abs() / eval_df['actual']
    print(eval_df.groupby('h')['APE'].mean())
 

In [ ]:
# Print one-step forecast MAPE
print('One step forecast MAPE: ', (mean_absolute_percentage_error(eval_df[eval_df['h'] 
== 't+1']['prediction'], 
eval_df[eval_df['h'] == 't+1']['actual']))*100, '%')
 
# Print multi-step forecast MAPE
print('Multi-step forecast MAPE: ', 
mean_absolute_percentage_error(eval_df['prediction'], eval_df['actual'])*100, '%')